## Intro

This is inspired by 
Article (likas2001probability) Likas, A. Probability density estimation using artificial neural networks Computer physics communications, Elsevier, 2001, 135, 167-175

But rather than estimating the working with a network, we will instead work with its derivitive.
This will let us replace their integration with a derivative.

Note that this method only works for compact supports



They use the PDF is given by $$p_h(x,p) = \dfrac{h(x,p)}{\int_S h(z,p) dz}$$
and in their case $h=N(x,p)$  a neural network with weight and bias parameters $p$.
Where $S$ is a compact support. (That means bounded)


But if instead we say $h=\frac{\partial N(x,p)}{\partial x}$,

then $$p_h(x,p) = \dfrac{h(x,p)}{\int_S h(z,p)}=\dfrac{\frac{\partial N(x,p)}{\partial x}}{N(max(S),p) - N(min(S), p)}$$

The denominator is ofcourse more complex for non-1D values of S.


The loss function given is the negative log-likelihood of the set of training samples $X$
$$L(p) = -\sum_{\forall x \in X} ln(h(x,p))  + |X| ln(\int_S h(z,p) dx)$$

Which befomes:

$$L(p) = -\sum_{\forall x \in X} log(\frac{\partial N(x,p)}{\partial x})  + |X|(ln(N(max(S),p)-N(min(S),p)) dx$$

In [1]:
using DensityEstimationML
function only(itr)
    state = start(itr)
    val,state = next(itr, state)
    @assert(done(itr,state))
    return val
end

only (generic function with 1 method)

In [2]:
using TensorFlow
using Distributions
using StatsBase
using StaticArrays

In [3]:
dataset = GenerateDatasets.Likas1()
data = original_sample(dataset)
est = NeuralDensityEstimator([64], approximate_support(dataset))

2017-09-21 16:34:05.339908: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.1 instructions, but these are available on your machine and could speed up CPU computations.
2017-09-21 16:34:05.339940: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.2 instructions, but these are available on your machine and could speed up CPU computations.
2017-09-21 16:34:05.339945: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX instructions, but these are available on your machine and could speed up CPU computations.
2017-09-21 16:34:05.563312: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:893] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2017-09-21 16:34:05.563927: I tensorflow/core/common_runtime/gpu/gpu_device.cc:940] Found device 0 with propert

pdf = <Tensor pdf:1 shape=unknown dtype=Float64>


DensityEstimationML.NeuralDensityEstimator{1}(Session(Ptr{Void} @0x00007f2cc7fddc30), <Tensor Group:1 shape=unknown dtype=Any>, <Tensor Group_2:1 shape=unknown dtype=Any>, <Tensor t:1 shape=(1, ?) dtype=Float32>, <Tensor pdf:1 shape=unknown dtype=Float64>)

In [4]:
pdf(est, data)

5000-element Array{Float64,1}:
 0.0416668
 0.0416668
 0.0416668
 0.0416665
 0.0416668
 0.0416668
 0.0416668
 0.0416668
 0.0416665
 0.0416665
 0.0416668
 0.0416665
 0.0416666
 ⋮        
 0.0416668
 0.0416667
 0.0416668
 0.0416668
 0.0416667
 0.0416668
 0.0416668
 0.0416666
 0.0416668
 0.0416665
 0.0416668
 0.0416667

In [5]:
liklpdf(est, data)

LoadError: [91mUndefVarError: liklpdf not defined[39m

In [5]:
@which pdf(est, DensityEstimationML.autostatic_array(data))

pdf(est::DensityEstimationML.NeuralDensityEstimator, ts::AbstractArray) in DensityEstimationML at /home/uniwa/students2/students/20361362/linux/.julia/v0.6/DensityEstimationML/src/estimator.jl:33

In [6]:
DensityEstimationML.autostatic_array(data) |> typeof <: StaticVector{5000}

true

In [7]:
StaticVector{5000} <: AbstractVector

true

In [22]:
function Distributions.pdf(est::NeuralDensityEstimator{1}, ts::StaticArrays.StaticArray{Tuple{1},Float64,1})
    println("hi")
end

In [19]:
DensityEstimationML.autostatic_array(data) |> typeof |> supertype

StaticArrays.StaticArray{Tuple{5000},Float64,1}

In [25]:
pdf(est, data)

LoadError: [91mUndefVarError: _pdf not defined[39m

In [ ]:
dataset = GenerateDatasets.Likas3()
data = original_sample(dataset)
est = NeuralDensityEstimator([64], approximate_support(dataset))
sess = est.sess
run(sess, est.pdf, Dict(est.t=>[0.1 0.1; 0.1 0.1; 0.3 0.6]'))

In [15]:
expand(:(x[end]))

:(getindex(x, (Base.endof)(x)))

In [10]:
gradients(gr["yt"], gr["t"])[1,:] |> get_shape

LoadError: [91mUndefVarError: gr not defined[39m

In [17]:
reshape(RowVector([1,2,3]), Val(2))

1×3 RowVector{Int64,Array{Int64,1}}:
 1  2  3

In [ ]:
println.(sort(gr |> keys |> collect))

In [6]:
using Plots
using IJulia

"""
Function returning a function that will display a running plot.
WARNING: Introducting or removing any variables is not supported.
And will silently error.
"""
function running_plot()
    epochs = Int[]
    record = Dict()
    function inner(epoch, vars::Associative)
        for (var, values) in vars
            value = only(values) #Incase it was an array
            past = get!(record, var) do
                typeof(value)[]
            end
            push!(past, value)
        end
        push!(epochs, epoch)
        
        IJulia.clear_output(true)
        plot(epochs, hcat(values(record)...); label=hcat(keys(vars)...), layout=length(vars)) |> IJulia.display       
    end
end


running_plot

In [11]:
function demonstration_plot(est, dataset, data, args...; kwargs...)
    X = minimum(approximate_support(dataset)) : 0.01 : maximum(approximate_support(dataset)) |> collect
    println("True loglikelihood      = $(loglikelihood(dataset, data))")
    println("Estimated loglikelihood = $(loglikelihood(est, data))")
    plot([X], [pdf(est,X), data],
        #xlims= approximate_support(dataset),
        xlims= (first(X), last(X)),
        seriestype = [:path :histogram],
        layout=(2,1),
        legend=false,
        nbins=[1  length(data)÷10],
        args...; kwargs...
    )
end

demonstration_plot (generic function with 1 method)

In [12]:
function demo(dataset, layers, epochs=20_000; max_conditioning_epochs=2000)
    data = original_sample(dataset)
    @show loglikelihood(dataset, data)
    est = NeuralDensityEstimator(layers, approximate_support(dataset))

    condition!(est; max_epochs = max_conditioning_epochs)
    println("Conditioning Done")
    Plots.gr()
    fit!(est, data; epochs=epochs, callback=running_plot())
    println("Fitting Done")
    plotly()
    demonstration_plot(est, dataset, data) |> IJulia.display
    
    est    
end

demo (generic function with 2 methods)

In [14]:
est=demo(GenerateDatasets.Likas1(), [64, 64, 256], 10_000)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2500 
 
 
 5000 
 
 
 7500 
 
 
 5 
 
 
 10 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 55.6177,175.813 57.9223,73.3418 60.2269,39.2384 62.5315,51.0876 64.8362,62.5834 67.1408,82.361 69.4454,92.5716 71.75,97.8157 74.0547,96.7348 76.3593,87.167 
 78.6639,67.8436 80.9686,45.7059 83.2732,28.2917 85.5778,16.5022 87.8824,11.811 90.1871,39.7215 92.4917,62.4718 94.7963,73.7773 97.1009,84.1335 99.4056,92.9629 
 101.71,97.5548 104.015,100.174 106.319,101.43 108.624,101.861 110.929,101.868 113.233,101.651 115.538,101.353 117.843,101.057 120.147,100.795 122.452,100.559 
 124.756,100.346 127.061,100.138 129.366,99.9282 131.67,99.7018 133.975,99.444 136.28,99.1397 138.584,98.7781 140.889,98.3222 143.193,97.7677 145.498,97.0872 
 147.803,96.274 150.107,95.3011 152.412,94.1708 154.717,92.872 157.021,91.4049 159.326,89.7523 161.63,87.8583 163.935,85.9142 166.24,85.2085 168.544,85.13 
 170.849,85.6873 173.154,87.1562 175.458,89.6786 177.763,93.4377 180.068,96.2763 182.372,98.095 184.677,99.6187 186.981,101.041 189.286,102.438 191.591,103.824 
 193.895,105.216 196.2,106.602 198.505,107.992 200.809,109.377 203.114,110.758 205.418,112.122 207.723,113.474 210.028,114.81 212.332,116.137 214.637,117.496 
 216.942,118.848 219.246,120.111 221.551,121.348 223.855,122.5 226.16,123.63 228.465,124.681 230.769,125.75 233.074,126.73 235.379,127.683 237.683,128.628 
 239.988,129.495 242.292,130.342 244.597,131.153 246.902,131.918 249.206,132.639 251.511,133.347 253.816,133.973 256.12,134.612 258.425,135.197 260.729,135.783 
 263.034,136.321 265.339,136.863 267.643,137.356 269.948,137.858 272.253,138.341 274.557,138.792 276.862,139.226 279.166,139.643 281.471,140.04 283.776,140.454 
 
 "/>
 
 
 
 
 ysmax 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2500 
 
 
 5000 
 
 
 7500 
 
 
 20000 
 
 
 30000 
 
<polyline clip-path="url(#clip03)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 352.157,11.811 354.462,113.079 356.766,135.291 359.071,139.22 361.375,140.448 363.68,141.055 365.985,141.5 368.289,142.207 370.594,143.648 372.899,145.691 
 375.203,147.852 377.508,149.777 379.812,151.302 382.117,152.465 384.422,153.55 386.726,155.74 389.031,158.181 391.336,160.595 393.64,163.095 395.945,165.623 
 398.249,167.669 400.554,169.251 402.859,170.285 405.163,170.942 407.468,171.4 409.773,171.705 412.077,171.899 414.382,172.028 416.686,172.12 418.991,172.189 
 421.296,172.243 423.6,172.288 425.905,172.328 428.21,172.364 430.514,172.397 432.819,172.428 435.123,172.459 437.428,172.49 439.733,172.521 442.037,172.554 
 444.342,172.588 446.647,172.626 448.951,172.668 451.256,172.716 453.56,172.773 455.865,172.845 458.17,172.935 460.474,173.049 462.779,173.192 465.084,173.353 
 467.388,173.511 469.693,173.663 471.998,173.832 474.302,174.066 476.607,174.268 478.911,174.43 481.216,174.568 483.521,174.686 485.825,174.788 488.13,174.877 
 490.435,174.957 492.739,175.028 495.044,175.091 497.348,175.148 499.653,175.198 501.958,175.242 504.262,175.281 506.567,175.314 508.872,175.343 511.176,175.369 
 513.481,175.39 515.785,175.41 518.09,175.427 520.395,175.442 522.699,175.456 525.004,175.469 527.309,175.481 529.613,175.493 531.918,175.504 534.222,175.515 
 536.527,175.526 538.832,175.536 541.136,175.548 543.441,175.56 545.746,175.573 548.05,175.588 550.355,175.604 552.659,175.62 554.964,175.637 557.269,175.653 
 559.573,175.67 561.878,175.687 564.183,175.703 566.487,175.72 568.792,175.736 571.096,175.752 573.401,175.767 575.706,175.783 578.01,175.798 580.315,175.813 
 
 "/>
 
 
 
 
 working_loss 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2500 
 
 
 5000 
 
 
 7500 
 
 
 -30000 
 
 
 -20000 
 
<polyline clip-path="url(#clip04)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 55.6177,375.813 57.9223,274.533 60.2269,252.315 62.5315,248.394 64.8362,247.171 67.14

Fitting Done
True loglikelihood      = -10549.508175739062
Estimated loglikelihood = -10843.065339653353


DensityEstimationML.NeuralDensityEstimator{1}(Session(Ptr{Void} @0x00007f2c980b4610), <Tensor Group:1 shape=unknown dtype=Any>, <Tensor Group_2:1 shape=unknown dtype=Any>, <Tensor t:1 shape=(1, ?) dtype=Float32>, <Tensor pdf:1 shape=unknown dtype=Float64>)

In [15]:
sort(component_weights(est))

256-element Array{Float32,1}:
 0.0       
 0.0       
 0.0       
 0.0       
 0.0       
 0.0       
 0.0       
 0.0       
 0.0       
 0.0       
 0.0       
 0.0       
 0.0       
 ⋮         
 0.00387415
 0.00472575
 0.00523693
 0.00628663
 0.00741258
 0.00745783
 0.00913192
 0.0546251 
 0.20398   
 0.472412  
 0.552228  
 1.91836   

In [16]:
demo(GenerateDatasets.Likas2(), [64,64], 20_000)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 5000 
 
 
 10000 
 
 
 15000 
 
 
 5 
 
 
 10 
 
 
 15 
 
 
 20 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 55.6177,175.813 56.7642,158.853 57.9107,115.068 59.0572,46.3254 60.2038,11.811 61.3503,14.4695 62.4968,21.4544 63.6433,27.8923 64.7898,32.9079 65.9364,35.1666 
 67.0829,36.3265 68.2294,38.4117 69.3759,41.1209 70.5225,43.9634 71.669,46.6051 72.8155,48.9649 73.962,51.1415 75.1085,53.2837 76.2551,55.5177 77.4016,57.9199 
 78.5481,60.512 79.6946,63.2698 80.8412,66.1374 81.9877,69.0436 83.1342,71.9104 84.2807,74.667 85.4273,77.3466 86.5738,80.05 87.7203,82.7697 88.8668,85.4998 
 90.0133,88.2084 91.1599,90.8762 92.3064,93.497 93.4529,96.0644 94.5994,98.5688 95.746,101.001 96.8925,103.35 98.039,105.605 99.1855,107.751 100.332,109.771 
 101.479,111.658 102.625,113.408 103.772,115.028 104.918,116.534 106.065,117.94 107.211,119.257 108.358,120.493 109.504,121.655 110.651,122.747 111.797,123.782 
 112.944,124.754 114.09,125.685 115.237,126.564 116.383,127.419 117.53,128.216 118.676,128.984 119.823,129.71 120.969,130.383 122.116,130.839 123.263,130.904 
 124.409,130.991 125.556,131.403 126.702,132.124 127.849,132.931 128.995,133.835 130.142,135.001 131.288,136.248 132.435,137.412 133.581,138.471 134.728,139.447 
 135.874,140.344 137.021,141.157 138.167,141.866 139.314,142.507 140.46,143.232 141.607,144.2 142.753,145.342 143.9,146.573 145.046,147.806 146.193,149.006 
 147.339,150.11 148.486,151.107 149.633,151.99 150.779,152.765 151.926,153.444 153.072,154.037 154.219,154.558 155.365,155.023 156.512,155.417 157.658,155.708 
 158.805,155.945 159.951,156.19 161.098,156.469 162.244,156.813 163.391,157.196 164.537,157.615 165.684,158.028 166.83,158.426 167.977,158.805 169.123,159.169 
 170.27,159.514 171.416,159.845 172.563,160.159 173.71,160.459 174.856,160.738 176.003,161.004 177.149,161.254 178.296,161.487 179.442,161.707 180.589,161.918 
 181.735,162.121 182.882,162.32 184.028,162.515 185.175,162.707 186.321,162.899 187.468,163.091 188.614,163.279 189.761,163.474 190.907,163.661 192.054,163.851 
 193.2,164.048 194.347,164.236 195.493,164.435 196.64,164.63 197.786,164.823 198.933,165.024 200.08,165.206 201.226,165.177 202.373,165.154 203.519,165.383 
 204.666,165.643 205.812,165.916 206.959,166.198 208.105,166.484 209.252,166.739 210.398,166.958 211.545,167.117 212.691,167.242 213.838,167.331 214.984,167.4 
 216.131,167.47 217.277,167.527 218.424,167.596 219.57,167.657 220.717,167.73 221.863,167.799 223.01,167.875 224.157,167.95 225.303,168.023 226.45,168.059 
 227.596,167.85 228.743,168.01 229.889,168.16 231.036,168.294 232.182,168.425 233.329,168.553 234.475,168.678 235.622,168.798 236.768,168.915 237.915,169.027 
 239.061,169.14 240.208,169.247 241.354,169.362 242.501,169.471 243.647,169.582 244.794,169.695 245.94,169.805 247.087,169.921 248.233,170.027 249.38,170.139 
 250.527,170.248 251.673,170.358 252.82,170.461 253.966,170.56 255.113,170.663 256.259,170.759 257.406,170.855 258.552,170.938 259.699,171.02 260.845,171.09 
 261.992,171.16 263.138,171.215 264.285,171.265 265.431,171.304 266.578,171.341 267.724,171.364 268.871,171.373 270.017,171.353 271.164,171.194 272.31,170.507 
 273.457,170.658 274.604,170.954 275.75,171.322 276.897,171.666 278.043,171.947 279.19,172.152 280.336,172.279 281.483,172.345 282.629,172.355 283.776,172.334 
 
 "/>
 
 
 
 
 ysmax 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 5000 
 
 
 10000 
 
 
 15000 
 
 
 10000 
 
 
 15000 
 
 
 20000 
 
<polyline clip-path="url(#clip03)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 352.157,11.811 353.303,113.437 354.45,150.211 355.596,159.775 356.743,162.483 357.89,163.752 359.036,164.53 360.183,165.138 361.329,165.766 362.476,166.448 
 363.622,167.087 364.769,167.667 365.915,168.203 367.062,168.702 368.208,169.162 369.355,169.585 370.501,169.973 371.64

Fitting Done
True loglikelihood      = -7148.443992586796
Estimated loglikelihood = -7168.291242831289


DensityEstimationML.NeuralDensityEstimator{1}(Session(Ptr{Void} @0x00007f2c9854ed30), <Tensor Group:1 shape=unknown dtype=Any>, <Tensor Group_2:1 shape=unknown dtype=Any>, <Tensor t:1 shape=(1, ?) dtype=Float32>, <Tensor pdf:1 shape=unknown dtype=Float64>)

In [17]:
demo(GenerateDatasets.MagdonIsmailAndAtiya(), [32], 10_000)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2500 
 
 
 5000 
 
 
 7500 
 
 
 2 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 42.4384,75.3492 44.8595,74.0185 47.2806,45.3722 49.7017,27.7179 52.1228,26.2001 54.544,15.3029 56.9651,11.811 59.3862,12.8628 61.8073,15.9292 64.2284,20.3582 
 66.6495,25.8541 69.0706,32.2614 71.4917,39.487 73.9128,47.452 76.3339,56.082 78.7551,65.2943 81.1762,74.9853 83.5973,85.0322 86.0184,95.2826 88.4395,105.57 
 90.8606,115.707 93.2817,125.521 95.7028,134.834 98.1239,143.507 100.545,151.405 102.966,158.399 105.387,163.94 107.808,167.728 110.229,170.316 112.651,171.993 
 115.072,173.043 117.493,173.744 119.914,174.224 122.335,174.562 124.756,174.808 127.177,174.983 129.598,175.113 132.019,175.059 134.441,174.983 136.862,174.905 
 139.283,174.826 141.704,174.749 144.125,174.679 146.546,174.626 148.967,174.594 151.388,174.592 153.809,174.619 156.231,174.675 158.652,174.755 161.073,174.851 
 163.494,174.946 165.915,175.03 168.336,175.114 170.757,175.188 173.178,175.249 175.599,175.299 178.021,175.353 180.442,175.385 182.863,175.424 185.284,175.453 
 187.705,175.484 190.126,175.506 192.547,175.531 194.968,175.547 197.389,175.567 199.811,175.586 202.232,175.598 204.653,175.614 207.074,175.624 209.495,175.638 
 211.916,175.647 214.337,175.659 216.758,175.672 219.179,175.677 221.601,175.691 224.022,175.699 226.443,175.713 228.864,175.724 231.285,175.729 233.706,175.739 
 236.127,175.743 238.548,175.751 240.969,175.76 243.391,175.76 245.812,175.768 248.233,175.772 250.654,175.776 253.075,175.782 255.496,175.784 257.917,175.789 
 260.338,175.789 262.759,175.794 265.181,175.799 267.602,175.798 270.023,175.803 272.444,175.807 274.865,175.805 277.286,175.809 279.707,175.813 282.128,175.813 
 
 "/>
 
 
 
 
 ysmax 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2500 
 
 
 5000 
 
 
 7500 
 
 
 800 
 
 
 825 
 
 
 850 
 
<polyline clip-path="url(#clip03)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 340.625,11.811 343.046,14.0789 345.467,24.3757 347.888,37.0725 350.31,47.9636 352.731,59.563 355.152,69.5491 357.573,76.8644 359.994,82.7405 362.415,87.6034 
 364.836,91.6889 367.257,95.1517 369.678,98.1047 372.099,100.635 374.521,102.815 376.942,104.704 379.363,106.352 381.784,107.803 384.205,109.096 386.626,110.263 
 389.047,111.33 391.468,112.321 393.889,113.256 396.311,114.149 398.732,115.012 401.153,115.854 403.574,116.68 405.995,117.492 408.416,118.291 410.837,119.081 
 413.258,119.87 415.679,120.663 418.101,121.465 420.522,122.277 422.943,123.101 425.364,123.937 427.785,124.784 430.206,126.318 432.627,127.632 435.048,128.998 
 437.469,130.409 439.891,131.836 442.312,133.241 444.733,134.59 447.154,135.859 449.575,137.044 451.996,138.148 454.417,139.185 456.838,140.169 459.259,141.113 
 461.681,142.027 464.102,142.917 466.523,143.787 468.944,144.642 471.365,145.483 473.786,146.309 476.207,147.119 478.628,147.921 481.049,148.706 483.471,149.479 
 485.892,150.238 488.313,150.984 490.734,151.717 493.155,152.439 495.576,153.148 497.997,153.845 500.418,154.532 502.839,155.208 505.261,155.874 507.682,156.531 
 510.103,157.178 512.524,157.819 514.945,158.453 517.366,159.089 519.787,159.742 522.208,160.446 524.629,161.211 527.051,161.978 529.472,162.774 531.893,163.529 
 534.314,164.262 536.735,164.973 539.156,165.666 541.577,166.344 543.998,167.006 546.419,167.652 548.841,168.294 551.262,168.921 553.683,169.539 556.104,170.148 
 558.525,170.747 560.946,171.339 563.367,171.923 565.788,172.5 568.209,173.069 570.631,173.631 573.052,174.187 575.473,174.735 577.894,175.278 580.315,175.813 
 
 "/>
 
 
 
 
 working_loss 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2500 
 
 
 5000 
 
 
 7500 
 
 
 -850 
 
 
 -825 
 
 
 -800 
 
<polyline clip-path="url(#clip04)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 42.4384,375.813 44.8595,373.549 47.2806,

Fitting Done
True loglikelihood      = -791.6688945841678


Estimated loglikelihood = -795.8466904538325


DensityEstimationML.NeuralDensityEstimator{1}(Session(Ptr{Void} @0x00007f2c98da3350), <Tensor Group:1 shape=unknown dtype=Any>, <Tensor Group_2:1 shape=unknown dtype=Any>, <Tensor t:1 shape=(1, ?) dtype=Float32>, <Tensor pdf:1 shape=unknown dtype=Float64>)

In [18]:
demo(Arcsine(1,4), [64,64], 20_000)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 5000 
 
 
 10000 
 
 
 15000 
 
 
 5 
 
 
 10 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 49.028,175.813 50.2077,160.132 51.3873,158.183 52.5669,158.403 53.7466,157.29 54.9262,154.907 56.1059,150.594 57.2855,142.956 58.4651,129.125 59.6448,104.012 
 60.8244,68.287 62.004,46.4959 63.1837,42.1471 64.3633,41.6049 65.5429,40.8543 66.7226,36.5018 67.9022,23.9577 69.0819,14.5594 70.2615,11.811 71.4411,11.9162 
 72.6208,12.5849 73.8004,13.3575 74.98,13.5368 76.1597,15.6116 77.3393,18.149 78.5189,21.2204 79.6986,24.2768 80.8782,27.6554 82.0579,31.033 83.2375,34.1388 
 84.4171,36.6437 85.5968,38.708 86.7764,40.9893 87.956,43.3646 89.1357,45.6854 90.3153,47.9126 91.4949,49.9553 92.6746,51.7634 93.8542,53.2239 95.0339,55.1833 
 96.2135,57.6931 97.3931,60.3304 98.5728,62.9116 99.7524,65.3404 100.932,67.5988 102.112,69.738 103.291,72.0062 104.471,74.6839 105.651,77.6169 106.83,80.1937 
 108.01,82.4946 109.189,84.9279 110.369,87.4828 111.549,90.0321 112.728,92.4987 113.908,94.8198 115.088,96.9675 116.267,98.9723 117.447,100.842 118.627,102.247 
 119.806,103.184 120.986,103.363 122.165,101.861 123.345,100.776 124.525,101.57 125.704,102.711 126.884,103.975 128.064,105.291 129.243,106.611 130.423,107.918 
 131.603,109.183 132.782,110.412 133.962,111.598 135.141,112.739 136.321,113.838 137.501,114.891 138.68,115.906 139.86,116.889 141.04,117.828 142.219,118.743 
 143.399,119.629 144.579,120.489 145.758,121.323 146.938,122.133 148.118,122.92 149.297,123.682 150.477,124.421 151.656,125.137 152.836,125.834 154.016,126.161 
 155.195,126.039 156.375,125.96 157.555,125.514 158.734,125.586 159.914,126.397 161.094,127.216 162.273,128.102 163.453,128.979 164.632,129.821 165.812,130.643 
 166.992,131.451 168.171,132.217 169.351,132.962 170.531,133.68 171.71,134.36 172.89,135.023 174.07,135.648 175.249,136.242 176.429,136.804 177.608,137.333 
 178.788,137.829 179.968,138.292 181.147,138.724 182.327,139.123 183.507,139.49 184.686,139.831 185.866,140.141 187.046,140.427 188.225,140.68 189.405,140.913 
 190.584,141.125 191.764,141.321 192.944,141.49 194.123,141.645 195.303,141.778 196.483,141.883 197.662,141.985 198.842,142.078 200.022,142.166 201.201,142.242 
 202.381,142.317 203.56,142.394 204.74,142.463 205.92,142.538 207.099,142.608 208.279,142.677 209.459,142.756 210.638,142.861 211.818,142.983 212.998,143.124 
 214.177,143.273 215.357,143.427 216.536,143.587 217.716,143.747 218.896,143.907 220.075,144.065 221.255,144.215 222.435,144.374 223.614,144.523 224.794,144.672 
 225.974,144.821 227.153,144.975 228.333,145.14 229.512,145.318 230.692,145.486 231.872,145.666 233.051,145.837 234.231,145.999 235.411,146.162 236.59,146.317 
 237.77,146.476 238.95,146.626 240.129,146.774 241.309,146.924 242.488,147.066 243.668,147.204 244.848,147.34 246.027,147.462 247.207,147.587 248.387,147.71 
 249.566,147.816 250.746,147.929 251.926,148.033 253.105,148.126 254.285,148.212 255.464,148.284 256.644,148.383 257.824,148.495 259.003,148.591 260.183,148.667 
 261.363,148.742 262.542,148.81 263.722,148.884 264.902,148.948 266.081,149.009 267.261,149.067 268.44,149.12 269.62,149.17 270.8,149.213 271.979,149.259 
 273.159,149.296 274.339,149.332 275.518,149.364 276.698,149.396 277.878,149.427 279.057,149.46 280.237,149.494 281.416,149.532 282.596,149.576 283.776,149.627 
 
 "/>
 
 
 
 
 ysmax 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 5000 
 
 
 10000 
 
 
 15000 
 
 
 5000 
 
 
 6000 
 
 
 7000 
 
<polyline clip-path="url(#clip03)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 345.567,11.811 346.747,72.1423 347.927,76.376 349.106,76.9499 350.286,77.6244 351.465,78.4202 352.645,79.3432 353.825,80.3987 355.004,81.5882 356.184,82.879 
 357.364,84.105 358.543,85.11 359.723,85.9885 360.903,86.8059 362.082,87.648 363.262,88.8279 364.441,90.9075 365.621,93.5083 366.801,96.07

Fitting Done
True loglikelihood      = -4276.673668996218
Estimated loglikelihood = -4657.898328963629


DensityEstimationML.NeuralDensityEstimator{1}(Session(Ptr{Void} @0x00007f2c9855dbb0), <Tensor Group:1 shape=unknown dtype=Any>, <Tensor Group_2:1 shape=unknown dtype=Any>, <Tensor t:1 shape=(1, ?) dtype=Float32>, <Tensor pdf:1 shape=unknown dtype=Float64>)

In [ ]:
dataset = GenerateDatasets.Likas3()
data = original_sample(dataset)
est = NeuralDensityEstimator([64], approximate_support(GenerateDatasets.Likas3()))

In [ ]:
@show loglikelihood(dataset, data)

In [ ]:
est = NeuralDensityEstimator([64], approximate_support(GenerateDatasets.Likas3()))

In [ ]:
condition!(est)

In [ ]:

fit!(est, data'; epochs=10_000, callback=running_plot())

In [ ]:

fit!(est, data; epochs=10_000, callback=running_plot())
println("Fitting Done")

In [ ]:
sess = est.sess
run(sess, est.pdf, Dict(est.t=>[0.1 0.1; 0.1 0.1; 0.3 0.6]'))

In [ ]:
sess = est.sess
run(sess, sess.graph["numerator"], Dict(est.t=>[0.1 0.1; 0.1 0.1; 0.3 0.6]'))

In [ ]:
data

In [ ]:
run(sess, gather(constant(data), 1))

In [ ]:
plotly()

In [ ]:
"""
    meshgrid(xs, ys, zfun)

Evalates `zfun(x,y)` at each point in `xs`, and `ys`.
Returns 3 vectors, a list of x points a list of y points and the value of z at that point.
"""
function meshgrid(xs, ys, zfun) 
    # There is a cute generalisation of this with a `@generated` function
    xpoints = eltype(xs)[]
    ypoints = eltype(ys)[]
    zpoints = typeof(zfun(xs[1],ys[1]))[]
    sizehint!.([xpoints, ypoints, zpoints], length(xs)*length(ys))
    for x in xs, y in ys
        push!(xpoints,x)
        push!(ypoints,y)
        push!(zpoints,zfun(x,y))
    end
    
    xpoints, ypoints, zpoints
end


In [ ]:
X=-0.1:0.01:0.3
Y=-0.1:0.01:0.3
scatter3d(meshgrid(X,Y, (x,y)->pdf(est, [x,y]))...)

In [ ]:
pdf(est, SMatrix{1,2}([0.1 0.1]))

In [ ]:
est

In [ ]:
size([0.1 0.1])

In [ ]:
?SArray